In [1]:
# Import all necessery python packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import re
import requests as req
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
import time
import geopy.distance
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import roman
import seaborn as sns
from fake_useragent import UserAgent

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, RidgeCV, Ridge, lars_path
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline

from IPython.display import Image

import pickle

%matplotlib inline

In [2]:
def loadpickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [3]:
df = loadpickle('./data/dataframe.pkl')

In [4]:
df_work = df.copy()

In [5]:
df_work.head()

,ID,SIZE,PRICE,ROOMS,FLOOR,ADDRESS,QUALITY,YEAR_BUILT,FLOORS_IN_BUILDING,LIFT,Balcony,ORIENTATION,VIEW,METRO_NEARBY,PARKING,DISTRICT,LATITUDE,LONGITUDE
0,23545892,39 m²,"31,9 millió Ft",1,földszint,"7. kerület, Dob utca 50.",közepes állapotú,1950 előtt,2,nincs megadva,NaN,dél,utcai,False,"utca, közterület - fizetős övezet",7,47.501126,19.065041
1,23652121,79 m²,"65,9 millió Ft",3,4,"6. kerület, Belső-Terézváros",felújított,1950 előtt,nincs megadva,nincs megadva,NaN,nincs megadva,nincs megadva,False,nincs megadva,6,47.509494,19.065323
2,23662271,75 m²,89 millió Ft,2 + 1 fél,1,"7. kerület, Belső-Erzsébetváros",újszerű,1950 előtt,4,van,NaN,délnyugat,utcai,False,nincs megadva,7,47.502627,19.077243
3,23662566,50 m²,"22,9 millió Ft",2,4,"3. kerület, Bécsi út",felújítandó,1950-1980 között,10,van,NaN,nyugat,utcai,False,"utca, közterület - fizetős övezet",3,47.532673,19.036491
4,23688806,32 m²,"12,5 millió Ft",1,3,"9. kerület, Haller utca",jó állapotú,NaN,3,nincs,NaN,nincs megadva,nincs megadva,False,"utca, közterület",9,47.476099,19.077198


In [6]:
df_work['SIZE'] = df['SIZE'].str[:-3].str.replace(" ","").astype('int')

In [7]:
df_work['SIZE']

0        39
1        79
2        75
3        50
4        32
         ..
18556    24
18557    90
18558    68
18559    84
18560    30
Name: SIZE, Length: 18561, dtype: int32

In [ ]:
def convertMilliard(string):
    price = string.str.split(' ')
    return price + " millió Ft"

In [8]:
df_work[(df_work['PRICE'].str.endswith('milliárd Ft'))]

,ID,SIZE,PRICE,ROOMS,FLOOR,ADDRESS,QUALITY,YEAR_BUILT,FLOORS_IN_BUILDING,LIFT,Balcony,ORIENTATION,VIEW,METRO_NEARBY,PARKING,DISTRICT,LATITUDE,LONGITUDE
97,25441982,1020,"1,2 milliárd Ft",52,nincs megadva,"7. kerület, Király utca",nincs megadva,NaN,nincs megadva,nincs megadva,NaN,nincs megadva,utcai,False,teremgarázs hely - benne van az árban,7,47.506331,19.070639
7807,30644644,624,"2,09 milliárd Ft",10 + 2 fél,4,"1. kerület, Vár",felújított,2001-2010 között,4,van,33 m²,nincs megadva,panorámás,False,teremgarázs hely - benne van az árban,1,47.501485,19.032512
10541,30933370,2222222,"222,22 milliárd Ft",222,nincs megadva,"7. kerület, Rákóczi út",nincs megadva,NaN,nincs megadva,nincs megadva,222219 m²,nincs megadva,nincs megadva,False,nincs megadva,7,47.499044,19.077575
11345,30990796,314,"1,06 milliárd Ft",5,2,"5. kerület, Honvéd utca 3.",felújított,NaN,nincs megadva,van,10 m²,nyugat,utcai,False,nincs megadva,5,47.505505,19.050383
13480,31129486,902,"1,35 milliárd Ft",45,nincs megadva,"8. kerület, Gyulai Pál utca",nincs megadva,NaN,nincs megadva,nincs megadva,NaN,nincs megadva,nincs megadva,False,nincs megadva,8,47.493980,19.067634


In [ ]:
df_work[(df_work['PRICE'].str.endswith('milliárd Ft'))] = df_work[(df_work['PRICE'].str.endswith('milliárd Ft'))].apply(lambda x: convertMilliard(x))

## BKK

In [10]:
stops = pd.read_csv('./data/bkk/stops.txt')
stop_times = pd.read_csv('./data/bkk/stop_times.txt')
trips = pd.read_csv('./data/bkk/trips.txt')
routes = pd.read_csv('./data/bkk/routes.txt')

C:\Developer\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
route_to_trip = trips[['route_id', 'trip_id']]
stop_to_trip = stop_times[['trip_id', 'stop_id']]

In [12]:
stops_filtered = stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']]

In [13]:
all_stops = pd.merge(stop_to_trip.join(route_to_trip.set_index('trip_id'), on='trip_id'), stops_filtered, how='left', on='stop_id')

In [14]:
all_stops = all_stops.drop(columns=['trip_id', 'stop_id']).drop_duplicates()

In [15]:
routes[routes['route_short_name'] == 'M2']

,agency_id,route_id,route_short_name,route_long_name,route_type,route_desc,route_color,route_text_color,route_sort_order
291,BKK,5200,M2,NaN,1,Déli pályaudvar M / Örs vezér tere M+H,E41F18,FFFFFF,1


In [16]:
all_stops[all_stops['route_id'] == '3010']

,route_id,stop_name,stop_lat,stop_lon
994186,3010,Bécsi út / Vörösvári út,47.546762,19.029194
994187,3010,Óbudai rendelőintézet,47.544878,19.034772
994188,3010,Flórián tér,47.541109,19.040538
994189,3010,Szentlélek tér H,47.539377,19.045937
994190,3010,Népfürdő utca / Árpád híd,47.536087,19.059532
...,...,...,...,...
1024615,3010,Fiastyúk utca,47.540159,19.085595
1024616,3010,Frangepán utca,47.533945,19.082177
1024617,3010,Béke tér,47.529376,19.080195
1024625,3010,Bécsi út / Vörösvári út,47.548489,19.028994
